<a href="https://colab.research.google.com/github/gukouk176-debug/colab2/blob/main/DataScience_09_ipynb_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第9回講義：分類2
+ 決定木
+ ランダムフォレスト

## 全講義共通初期設定
+ 警告の非表示(実装時は非推奨)
+ numpy pandas小数点以下桁数の表示設定
+ pandas全データ表示設定
+ Google driveへの接続

In [ ]:
# ワーニングを非表示にする
# この設定は不都合が見えなくなる為、お勧めしない
# 今回は教育資料用に、出力を簡素化する為に利用する
import warnings
warnings.simplefilter('ignore')

# モジュールの読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 小数点以下桁数の表示設定
np.set_printoptions(precision = 3)
pd.options.display.precision = 3

# pandasの全データ表示設定
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
#google driveに接続
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
#google driveと接続できたかを確認
!ls drive/MyDrive/DataScience

# 決定木モデル
アヤメデータに決定木モデルを適用する

In [ ]:
#アヤメのデータに決定木モデルを適用する
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import tree

#ライブラリpandasを使ったcsvデータの読み込み,sepで区切り記号を設定
ayame_df = pd.read_csv("/content/drive/MyDrive/DataScience/iris.csv", sep=",")

ayame_df.info()

In [ ]:
ayame_df.head()

In [ ]:
#説明変数、目的変数の設定

#X = ayame_df.iloc[:,[0,1,2,3]]
#y = ayame_df.iloc[:,[4]]

X = ayame_df.drop('species', axis=1)
y = ayame_df['species']

print(X.head())
print(y.head())

In [ ]:
#テストデータとトレーニングデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None )

#説明変数の標準化
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

#決定木モデルの生成と学習
tree_model = tree.DecisionTreeClassifier(max_depth=3)
tree_model.fit(X_train_std,y_train)

#トレーニングデータによるフィッティング結果とテストデータによる学習モデルの精度の検証
print('正解率（train):  {:.4f}'.format(tree_model.score(X_train_std, y_train)))
print('正解率（test):  {:.4f}'.format(tree_model.score(X_test_std, y_test)))

### 演習1．自動車価格データに対して決定木モデルを適用

In [ ]:
#　自動車価格データについて，決定木モデルを用いて prive_over_10K の分類を行う













# ランダムフォレスト

In [ ]:
#ランダムフォレストを用いた機械学習
from sklearn.ensemble import RandomForestClassifier

#ランダムフォレストの生成と学習
rf = RandomForestClassifier(max_depth=3)
rf.fit(X_train_std,y_train)

#トレーニングデータによるフィッティング結果とテストデータによる学習モデルの精度の検証
print('正解率（train):  {:.4f}'.format(rf.score(X_train_std, y_train)))
print('正解率（test):  {:.4f}'.format(rf.score(X_test_std, y_test)))

### 演習２．自動車価格データに対してランダムフォレストを適用

In [ ]:
# 自動車価格データについて，ランファムフォレストを用いて prive_over_10K の分類を行う．
# 演習１の決定木の結果と比べてみる













